## Descriptors

A descriptor lets you customize what should be done when you refer to an attribute on an object.

The descriptor classes are based on three special methods that form the **descriptor protocol**:

* setter: __set__
* getter: __get__
* __delete__: called when `del` is invoked

Methods of this protocol are infact called by the object's special `__getattribute__()` method on every attribute lookup. Whenever such a lookup is performed, either by using dotted notation or by using getattr, the `__getattribute__()` is implicitly invoked and it looks for an attribute in the following order.

* data descriptor
* `__dict__`
* non-data descriptor


In [17]:
class RevealAccess:
    def __init__(self, initval=None, name='var'):
        self.val = initval
        self.name = name
    
    def __get__(self, obj, objtype):
        print('retrieving', self.name)
        return self.val
    
    def __set__(self, obj, val):
        print('updating', self.name)
        self.val = val
        
    def __delete__(self, obj):
        print('deleting', self.name)
        
class MyClass:
    x = RevealAccess(10, 'var "x"')
    y = 5

In [18]:
m = MyClass()
m.x

retrieving var "x"


10

In [19]:
m.x = 20

updating var "x"


In [20]:
m.y

5

In [21]:
del m.x

deleting var "x"


## functions are non-data descriptors

In [22]:
f = lambda a, b: a + b

In [24]:
hasattr(f, '__get__'), hasattr(f, '__set__')

(True, False)

## lazy-evaluated attributes


In [28]:
class InitOnAccess:
    def __init__(self, kls, *args, **kwargs):
        self.kls = kls
        self.args = args
        self.kwargs = kwargs
        self._initialized = None
        
    def __get__(self, instance, owner):
        if self._initialized is None:
            self._initialized = self.kls(*self.args, **self.kwargs)
            print('initialized')
        else:
            print('cached')
        return self._initialized
    

class MyClass:
    lazy_val = InitOnAccess(list, [1, 2, 3])

In [29]:
m = MyClass()
m.lazy_val

initialized


[1, 2, 3]

In [30]:
m.lazy_val

cached


[1, 2, 3]